In [6]:
import numpy as np
import heapq

class Node:
    """A class to represent a node in the search tree"""
    def __init__(self, level, path, reduced_matrix, cost_bound):
        self.level = level
        self.path = path
        self.reduced_matrix = reduced_matrix
        self.cost_bound = cost_bound
    
    def __lt__(self, other):
        return self.cost_bound < other.cost_bound  # Priority queue sorting


class BranchBoundSolver:
    def __init__(self, filename):

        distance_matrix = []

        with open(filename, 'r') as f:
            lines = f.readlines()
            read_distances = False  

            for line in lines: 
                line = line.strip()
                line = line.replace(':', ' ')

                if line.startswith('EOF'):
                    break
                elif line.startswith('EDGE_WEIGHT_SECTION'):
                    read_distances = True
                elif read_distances:
                    elements = line.split()
                    row = [int(e) for e in elements]
                    row = [np.inf if e == 0 else e for e in row]
                    distance_matrix.append(row)

        self.graph = distance_matrix
        self.n = len(distance_matrix)
    
    def reduce_matrix(self):
        reduced_matrix = self.graph.copy()
    
        # Step 1: Row Reduction
        row_min = np.min(reduced_matrix, axis=1)
        row_min[row_min == np.inf] = 0  # Ignore infinity values
        reduced_matrix -= row_min[:, np.newaxis]

        # Step 2: Column Reduction
        col_min = np.min(reduced_matrix, axis=0)
        col_min[col_min == np.inf] = 0
        reduced_matrix -= col_min

        reduction_cost = np.sum(row_min) + np.sum(col_min)

        return reduced_matrix, reduction_cost

    def branch_and_bound_tsp(self):
        
        initial_matrix, lower_bound = self.reduce_matrix()
    
        pq = []
        best_cost = float('inf')
        best_path = []

        root = Node(level=0, path=[0], reduced_matrix=initial_matrix, cost_bound=lower_bound)
        heapq.heappush(pq, root)

        while pq:
            current_node = heapq.heappop(pq)

            if current_node.level == self.n - 1:
                final_cost = current_node.cost_bound + self.graph[current_node.path[-1]][0]
                if final_cost < best_cost:
                    best_cost = final_cost
                    best_path = current_node.path + [0]
                continue

            last_city = current_node.path[-1]
            for next_city in range(self.n):
                if next_city not in current_node.path:
                    new_path = current_node.path + [next_city]
                    new_matrix = current_node.reduced_matrix.copy()

                    new_matrix[last_city, :] = np.inf
                    new_matrix[:, next_city] = np.inf
                    new_matrix[next_city, 0] = np.inf

                    new_matrix, new_cost = self.reduce_matrix()
                    new_bound = current_node.cost_bound + self.graph[last_city][next_city] + new_cost

                    if new_bound < best_cost:
                        new_node = Node(level=current_node.level + 1, path=new_path, 
                                        reduced_matrix=new_matrix, cost_bound=new_bound)
                        heapq.heappush(pq, new_node)

        return best_path, best_cost

In [7]:
filename = "../tsplib_converted_instances/01_small/gr17_converted.tsp"

# Solve using 2-Opt 
branch_bound_solver = BranchBoundSolver(filename)
ch_cost, ch_path = branch_bound_solver.branch_and_bound_tsp()

print("\nChristofides-Solver - Minimum Cost:", ch_path)
print("\nPath: ", ch_cost)

KeyboardInterrupt: 